In [95]:
import openai
import streamlit as st
import pandas as pd
from shroomdk import ShroomDK
import config
from subgrounds import Subgrounds


In [96]:
#openai.api_key = openai_key
sdk = ShroomDK(config.shroomdk_key)



/var/folders/hj/lyy3jr1n31l00lc7kdsdnqzc0000gn/T/ipykernel_63359/2612213174.py:2: ResourceWarning: unclosed <ssl.SSLSocket fd=79, family=2, type=1, proto=0, laddr=('192.168.1.160', 57554), raddr=('168.220.87.95', 443)>
  sdk = ShroomDK(config.shroomdk_key)


In [97]:
my_address = "0x52F58796dA5B8D462b4FBE63dc3414D42A9776B9"
sql = f"""
    SELECT 
        nft_address, 
        mint_price_eth, 
        mint_price_usd 
    FROM ethereum.core.ez_nft_mints 
    WHERE nft_to_address = LOWER('{my_address}')
    """

In [98]:
query_result_set = sdk.query(sql)

/Users/dannyhughes/opt/anaconda3/envs/venv/lib/python3.11/site-packages/shroomdk/api.py:115: ResourceWarning: unclosed <ssl.SSLSocket fd=79, family=2, type=1, proto=0, laddr=('192.168.1.160', 57788), raddr=('168.220.87.95', 443)>
  self.__session = http


In [99]:
query_result_set

QueryResultSet(query_id=None, status='finished', columns=['NFT_ADDRESS', 'MINT_PRICE_ETH', 'MINT_PRICE_USD'], column_types=['string', 'number', 'number'], rows=[['0x6b1bde3d849c8f1bf2cf036613b04e5840769ac2', 0, 0], ['0x75524e4ca71ca52d1a82a997bfb7f59f7e6541d4', 0, 0], ['0x283501f44e60b412ebd92f9d88989c4d6f4e7726', 0.1, 235.96], ['0x7723e7257662df7702381474eec08d43104110c1', 0.0069, 11.33]], run_stats=QueryRunStats(started_at=datetime.datetime(2023, 3, 7, 23, 16, 25), ended_at=datetime.datetime(2023, 3, 7, 23, 16, 26), elapsed_seconds=1, record_count=4), records=[{'nft_address': '0x6b1bde3d849c8f1bf2cf036613b04e5840769ac2', 'mint_price_eth': 0, 'mint_price_usd': 0}, {'nft_address': '0x75524e4ca71ca52d1a82a997bfb7f59f7e6541d4', 'mint_price_eth': 0, 'mint_price_usd': 0}, {'nft_address': '0x283501f44e60b412ebd92f9d88989c4d6f4e7726', 'mint_price_eth': 0.1, 'mint_price_usd': 235.96}, {'nft_address': '0x7723e7257662df7702381474eec08d43104110c1', 'mint_price_eth': 0.0069, 'mint_price_usd': 11.

In [100]:
for record in query_result_set.records:
    nft_address = record['nft_address']
    mint_price_eth = record['mint_price_eth']
    mint_price_usd = record['mint_price_usd']
    print(f"${nft_address} minted for {mint_price_eth}ETH (${mint_price_usd})")

$0x6b1bde3d849c8f1bf2cf036613b04e5840769ac2 minted for 0ETH ($0)
$0x75524e4ca71ca52d1a82a997bfb7f59f7e6541d4 minted for 0ETH ($0)
$0x283501f44e60b412ebd92f9d88989c4d6f4e7726 minted for 0.1ETH ($235.96)
$0x7723e7257662df7702381474eec08d43104110c1 minted for 0.0069ETH ($11.33)


In [101]:
sg = Subgrounds()


In [102]:
quickswap = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/quickswap-polygon')


In [118]:
quickswap_swaps = quickswap.Query.swaps(orderBy = quickswap.Query.swaps.timestamp, orderDirection = 'desc')

In [120]:
quickswap_df = sg.query_df([
    quickswap_swaps.timestamp,
    quickswap_swaps.pool.name,
    quickswap_swaps.amountInUSD

])

2023-03-07 17:46:38.971 INFO    subgrounds: client.query: url = https://api.thegraph.com/subgraphs/name/messari/quickswap-polygon, variables = {'first0': 100, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  x9fdac392921bc368: swaps(first: $first0, skip: $skip0, orderBy: timestamp, orderDirection: desc, where: {}) {
    timestamp
    pool {
      name
    }
    amountInUSD
    id
  }
}


In [124]:
quickswap_df['swaps_timestamp'] = pd.to_datetime(quickswap_df['swaps_timestamp'], unit='s')

In [126]:
quickswap_df

,swaps_timestamp,swaps_pool_name,swaps_amountInUSD
0,2023-03-07 23:45:28,QuickSwap Wrapped Matic/(PoS) Tether USD,1.386862
1,2023-03-07 23:45:28,QuickSwap Bombcrypto Coin/(PoS) Tether USD,1.390940
2,2023-03-07 23:45:28,QuickSwap Wombat/USD Coin (PoS),1.069550
3,2023-03-07 23:45:28,QuickSwap Wrapped Matic/USD Coin (PoS),1.072768
4,2023-03-07 23:45:26,QuickSwap (PoS) Tether USD/PlanetIX,21.233240
...,...,...,...
95,2023-03-07 23:42:44,QuickSwap Wrapped Matic/Nitro (PoS),33.392230
96,2023-03-07 23:42:44,QuickSwap Polinate (PoS)/Wrapped Ether,8.911172
97,2023-03-07 23:42:44,QuickSwap Bombcrypto Coin/(PoS) Tether USD,0.660523
98,2023-03-07 23:42:36,QuickSwap Wrapped Matic/Sunflower Land,1.002394
